In [1]:
%matplotlib inline
import pyro
from math import pi
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from pyro.optim import Adam
import pyro.distributions as dist
from torch.distributions import constraints
from tqdm import tqdm
import seaborn as sns
from matplotlib import animation, rc
from IPython.display import HTML
import torch.nn as nn
from functools import partial
import pandas as pd
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import EmpiricalMarginal, SVI, Trace_ELBO, TracePredictive
import scipy.stats

pyro.set_rng_seed(1)
pyro.enable_validation(True)

In [2]:
def normalize_class(data_y):
    classes = list(unique_classes)
    print(classes)
    for i in range(len(data_y)):
        data_y[i] = int(classes.index(data_y[i]))
    return data_y

In [3]:
dataset = pd.read_csv('iris.csv', sep=',', header=[0])
data_x = torch.tensor(dataset.iloc[:,0:-1].values)
data_y = dataset.iloc[:,-1]
class_prob = data_y.value_counts(normalize = True)
unique_classes = np.unique(data_y)
class_amount = len(unique_classes)
atr_amount = data_x.shape[1]
data_y = torch.tensor(normalize_class(data_y)) #.unsqueeze(-1)
print(data_y)

['setosa', 'versicolor', 'virginica']


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])


In [4]:
for i in range(1, 10):
    print(pyro.sample('test', dist.Normal(torch.zeros(1, 1), 1.).independent(1)))

tensor([[0.6614]])
tensor([[0.2669]])
tensor([[0.0617]])
tensor([[0.6213]])
tensor([[-0.4519]])
tensor([[-0.1661]])
tensor([[-1.5228]])
tensor([[0.3817]])
tensor([[-1.0276]])


In [19]:
dist.Beta(torch.zeros(1, 1), torch.tensor(1.0))

ValueError: The parameter concentration has invalid values

In [ ]:
def model(atr, claz):
    #print(claz)
    class_prob = pyro.sample('class_prob', dist.Beta(torch.tensor(1.0), torch.tensor(1.0)).expand([class_amount]).independent(1))
    #print(class_prob)
    norm_class_prob = class_prob/torch.sum(class_prob)
    print('norm',norm_class_prob)
    #print(norm_class_prob)
    with pyro.plate('class_prob_loop', len(claz)):
        pyro.sample('obs', dist.Categorical(probs=norm_class_prob), obs=claz)
    

def guide(atr, claz):
    class_prob = pyro.param('class', torch.rand(1, class_amount)/class_amount)
    norm_class_prob = class_prob/torch.sum(class_prob)
    print(norm_class_prob)
    
    pyro.sample('class_prob', dist.Categorical(probs=norm_class_prob))
    
    #means = pyro.param('mean', pyro.distributions.Normal(torch.zeros(class_amount, atr_amount), 1.0))
    #stds = pyro.param('std', pyro.distributions.LogNormal(torch.zeros(class_amount, atr_amount), 0.1), constraint=constraints.positive)
    
    
guide(data_x, data_y)
    


pyro.clear_param_store()

def train(X, Y):
    pyro.clear_param_store()
    num_iterations=1000
    optim = pyro.optim.Adam({"lr": 0.05})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(X))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X, Y)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

train(data_x, data_y)

#pyro.params
    

  0%|          | 0/1000 [00:00<?, ?it/s]

In [6]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

In [5]:
data = torch.cat((torch.zeros(2691), torch.ones(7423 - 2691)))

def model(data):
    alpha = torch.tensor(6.0)
    beta = torch.tensor(10.0)
    wroc_probs = pyro.sample('wroc_probs', dist.Beta(alpha, beta).expand([2]).independent(1))
    norm_wroc_probs = wroc_probs/torch.sum(wroc_probs)
    with pyro.iarange('data_loop', len(data)):
        pyro.sample('obs', dist.Categorical(probs=norm_wroc_probs), obs=data)
        
def guide(data):
    alphas = pyro.param('alphas', torch.tensor(15.).expand(2), constraint=constraints.positive)
    betas = pyro.param('betas', torch.tensor(26.).expand(2), constraint=constraints.positive) 

    pyro.sample('wroc_probs', dist.Beta(alphas, betas).independent(1))
    
    
def train(data):
    pyro.clear_param_store()
    num_iterations=1000
    optim = pyro.optim.Adam({"lr": 0.05})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(data))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(data)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

#train(data)

In [4]:
torch.set_default_dtype(torch.float64)

class NaiveBayes(nn.Module):
    def __init__(self, data_x, data_y):
        super(self.__class__, self).__init__()
        self.data_x = data_x
        self.data_y = data_y
        
        #print(data_y)
        self.unique_classes = np.unique(data_y)
        self.class_amount = len(np.unique(data_y))
        self.atr_amount = data_x.shape[1]
        self.shape = (self.class_amount, self.atr_amount)
        
        #self.class_prob = self.data_y.value_counts(normalize = True)
        self.register_parameter("classes", nn.Parameter(torch.ones(self.class_amount)/self.class_amount))
        self.register_parameter("locs", nn.Parameter(torch.zeros(self.shape)))
        self.register_parameter("scales", nn.Parameter(torch.ones(self.shape)))
        
    def predict(self, x):
        item_prob = []
        for claz_i in range(self.class_amount):
            claz_prob = self.classes[claz_i]
            #print('1', claz_prob)
            for atr_i in range(self.atr_amount):
                    #print(claz_prob)
                mean = self.locs[claz_i][atr_i].detach().numpy()
                std = self.scales[claz_i][atr_i].detach().numpy()
                print(mean, std)
                claz_prob *= (1/(np.sqrt(2*np.pi*(std ** 2)))) * (np.e ** (-((x[atr_i]-torch.tensor(mean)) ** 2)/(2*std ** 2)))
            #print('2', claz_prob)
            item_prob.append(claz_prob.item())
            print('3', item_prob)
        return torch.tensor(item_prob)
        
        
        
    def forward(self, x):
        x = x[:,np.newaxis,:] #.double()

        v = (torch.sum(- 0.5 * torch.log(1/(2. * pi * self.scales))
                - (x - self.locs.double())**2 / torch.abs(self.scales) / 2, dim=-1)
                     ) #+ torch.log(self.classes))
        #print ("org ", v, "\n\n\n")
        
        #print("ret ", ret, '\n\n')
        #print(torch.tensor(ret))
        #print(v, v.shape)
        
        return v
    
        ret = []
        for item in x:
            #print(item)
            item_prob = []
            for claz_i in range(self.class_amount):
                claz_prob = self.classes[claz_i]
                for atr_i in range(self.atr_amount):
                    #print(claz_prob)
                    mean = self.locs[claz_i][atr_i].item()
                    std = self.scales[claz_i][atr_i].item()
                    print(mean, std)
                    claz_prob *= (1/(np.sqrt(2*np.pi*(std ** 2)))) * (np.e ** (-((item[atr_i]-mean) ** 2)/(2*std ** 2)))
                    
                    #p *= (1/(np.sqrt(2*np.pi*(std ** 2)))) * (np.e ** (-((element-mean) ** 2)/(2*std ** 2)))
                
                item_prob.append(claz_prob.item())
                print('item', item_prob)
            #print(ret)
            ret.append(item_prob)
            #print('aff',ret)
        
        print(ret)
        ret = torch.tensor(ret)
        return v
        #print(x)
        #print(self.scales)
        #print(self.locs)
        #print(self.classes)
        
    
    def model(self, data_x, data_y):
        #print(data_y)
        #data_y = data_y.unsqueeze(-1)
        #print(data_y)
        means = pyro.distributions.Normal(loc=torch.zeros_like(self.locs), scale=torch.ones_like(self.scales)).to_event(2)
        variances = pyro.distributions.LogNormal(loc=torch.ones_like(self.scales), scale=torch.ones_like(self.scales)).to_event(2)
        classes_priors = pyro.distributions.Dirichlet(concentration=torch.ones(self.class_amount))
        #classes_priors = classes_priors/torch.sum(classes_priors)
        
        priors = {'locs': means, 'scales': variances, "classes": classes_priors}
        lifted_module = pyro.random_module("module", self, priors)
        lifted_model = lifted_module()
        
        with pyro.plate("map", len(data_x)):
            class_prob = lifted_model(data_x)
            #pyro.sample("obs_cl",pyro.distributions.Categorical(probs=lifted_model.classes),obs=data_y)
            pyro.sample("obs",pyro.distributions.Categorical(logits=class_prob),obs=data_y)
            return class_prob
        
        """
        with pyro.plate('classes', self.class_amount):
            classes = pyro.sample("class", pyro.distributions.Categorical(lifted_model.classes), obs=data_y)
            with pyro.plate("map", len(data_x)):
                class_prob = lifted_model(data_x)
                print('class prob' , class_prob)
            #    pyro.sample("obs",pyro.distributions.Categorical(probs=lifted_model.classes),obs=data_y)
                pyro.sample("obs",pyro.distributions.Categorical(logits=class_prob),obs=data_y)
                return class_prob"""
        
    def guide(self, data_x, data_y):
        means_m = pyro.param("means_m",torch.ones_like(self.locs))
        means_s = pyro.param("means_s",torch.ones_like(self.locs), constraint=constraints.positive)
        variances_c = pyro.param("variances_a", torch.ones_like(self.scales), constraint=constraints.positive)
        variances_r = pyro.param("variances_b", torch.ones_like(self.scales), constraint=constraints.positive)
        classes_priors_c = pyro.param("classes_priors_c", torch.ones(self.class_amount)/self.class_amount, constraint=constraints.positive)
        classes_priors_c = classes_priors_c/torch.sum(classes_priors_c)
        
        locs = pyro.distributions.Normal(loc=means_m, scale=means_s).independent(2)
        scales = pyro.distributions.LogNormal(loc=variances_c,scale=variances_r).independent(2)
        classes = pyro.distributions.Dirichlet(concentration=classes_priors_c)
        
        
        
        priors = {'locs': locs, 'scales': scales, "classes": classes}
        lifted_module = pyro.random_module("module", self, priors)
        return lifted_module()
    
    def train(self, data_x, data_y):
        pyro.clear_param_store()
        num_iterations=10000
        optim = pyro.optim.Adam({"lr": 0.05})
        svi = pyro.infer.SVI(self.model, self.guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(data_x))
        losses = list()
        t=tqdm(range(num_iterations))
        for j in t:
            loss = svi.step(data_x, data_y)
            losses.append(loss)
            t.set_postfix(loss=loss)
        return (svi, losses)
    
naive_bayes = NaiveBayes(data_x, data_y)
svi, losses = naive_bayes.train(data_x, data_y)


100%|██████████| 10000/10000 [02:21<00:00, 70.67it/s, loss=77.9]


In [36]:


(- 0.5 * torch.log(1/(2. * pi * data_x)) + data_x).shape

torch.Size([150, 4])

In [34]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))
    
print(data_x[-3:])

means_m tensor([[ 0.5949,  0.6524, -1.0814, -0.1030],
        [ 0.6893, -0.2362,  1.6876,  0.1955],
        [ 0.4144, -1.4858,  0.6019,  2.8829]], requires_grad=True)
means_s tensor([[0.6529, 1.0211, 0.1759, 0.7522],
        [0.6437, 0.0811, 0.1085, 0.2129],
        [0.6312, 0.0780, 0.7963, 0.2242]], grad_fn=<AddBackward0>)
variances_a tensor([[2.3681e+00, 2.1530e+00, 2.3160e-04, 1.1421e+00],
        [2.8310e+00, 2.1186e-02, 6.6174e-04, 1.1269e-03],
        [1.9714e+00, 8.8332e-03, 2.6977e+00, 3.9348e-04]],
       grad_fn=<AddBackward0>)
variances_b tensor([[0.2943, 0.4714, 0.0889, 0.6801],
        [0.1282, 0.0572, 0.0798, 0.3421],
        [0.1442, 0.0395, 0.2421, 0.3204]], grad_fn=<AddBackward0>)
classes_priors_c tensor([0.2319, 0.3249, 0.4041], grad_fn=<AddBackward0>)
tensor([[6.5000, 3.0000, 5.2000, 2.0000],
        [6.2000, 3.4000, 5.4000, 2.3000],
        [5.9000, 3.0000, 5.1000, 1.8000]])


In [37]:
model = naive_bayes.guide(None, None)
res = model(data_x)
for i in range(150):
    values, indices = res[i].max(0)
    if(indices.item()==)
    print(, )

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 1
1 1
2 1
1 1
2 1
1 1
1 1
1 1
1 1
2 1
1 1
1 1
1 1
1 1
1 1
2 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2
2 2


In [19]:
def wrapped_model(data_x, data_y):
    model_result=probabilistic_model(data_x, data_y)
    pyro.sample("prediction", pyro.distributions.Delta(model_result))
    
posterior = svi.run(data_x, data_y)

trace_pred = TracePredictive(wrapped_model,
                             posterior,
                             num_samples=1)

In [31]:

model = naive_bayes.guide(None, None)

print((model(data_x)))

tensor([[3.2065e-17, 4.6777e-18, 2.2602e-05],
        [3.4421e-15, 2.8109e-17, 3.2038e-05],
        [1.2497e-13, 1.9683e-16, 3.6581e-05],
        [8.0041e-13, 2.3486e-16, 3.2419e-05],
        [2.2633e-16, 1.0916e-17, 2.2846e-05],
        [2.1371e-20, 1.8381e-19, 1.0562e-05],
        [5.7566e-13, 4.1603e-16, 3.1928e-05],
        [2.8567e-16, 8.0975e-18, 2.2447e-05],
        [2.8730e-11, 1.6324e-15, 4.4164e-05],
        [3.0177e-15, 1.4728e-17, 2.7078e-05],
        [2.9782e-20, 2.2300e-19, 1.5086e-05],
        [1.4391e-14, 3.0880e-17, 2.2258e-05],
        [2.3774e-14, 4.8123e-17, 3.3642e-05],
        [1.2961e-10, 6.3027e-15, 5.8852e-05],
        [9.6089e-25, 1.2642e-20, 1.3506e-05],
        [6.5534e-24, 2.0657e-20, 8.4525e-06],
        [2.3378e-20, 7.1859e-19, 1.7117e-05],
        [3.2043e-17, 6.2273e-18, 2.2753e-05],
        [1.5570e-23, 8.6697e-21, 9.0989e-06],
        [2.1026e-17, 4.2355e-18, 1.7398e-05],
        [4.1775e-20, 1.1480e-19, 1.3477e-05],
        [2.4051e-17, 5.6668e-18, 1